- https://www.reddit.com/r/flask/comments/gyf22b/if_you_are_using_flask_you_should_start_using/
- https://github.com/tiangolo/fastapi/issues/584
- https://stackoverflow.com/questions/53548127/post-numpy-array-with-json-to-flask-app-with-requests
- https://towardsdatascience.com/why-we-switched-from-flask-to-fastapi-for-production-machine-learning-765aab9b3679
- https://pypi.org/project/schema/
- https://medium.com/grammofy/testing-your-python-api-app-with-json-schema-52677fe73351
- https://restfulapi.net/json-schema/
- Pedantic Configuration Management with Pydantic : https://rednafi.github.io/digressions/python/2020/06/03/python-configs.html
- How we validate input data using pydantic:https://datascience.statnett.no/2020/05/11/how-we-validate-data-using-pydantic/
- How to parse list of models with Pydantic : https://stackoverflow.com/questions/55762673/how-to-parse-list-of-models-with-pydantic

In [157]:
import requests
import json
from datetime import datetime
import json
import pickle

# data validate modules
from typing import List, Optional
from pydantic import BaseModel,parse_obj_as,ValidationError
from marshmallow import Schema, fields
from marshmallow import ValidationError


import numpy as np
import scipy as sp
import pandas as pd 
import matplotlib.pyplot as plt 
from matplotlib import image

from api.app import create_app
from api.config import DevelopmentConfig,TestingConfig
from api.controller import prediction_app


## Flaks App Context

In [6]:
application = create_app(config_object=TestingConfig)

In [3]:
client=application.app_context().app.test_client()

In [4]:
client.get()

<Response streamed [404 NOT FOUND]>

In [51]:
s=client.get("/version")
s.get_json()

{'api_version': '0.2.0', 'model_version': '0.1'}

In [9]:
s=pd.read_csv("test.csv")[:1]
#s=pd.DataFrame({"Id":[2],"Street":["OASD"]})

In [11]:
j=s.to_json(orient="records")
j

'[{"Id":"1461","MSSubClass":20,"MSZoning":"RH","LotFrontage":80.0,"LotArea":11622,"Street":"Pave","Alley":null,"LotShape":"Reg","LandContour":"Lvl","Utilities":"AllPub","LotConfig":"Inside","LandSlope":"Gtl","Neighborhood":"NAmes","Condition1":"Feedr","Condition2":"Norm","BldgType":"1Fam","HouseStyle":"1Story","OverallQual":5,"OverallCond":6,"YearBuilt":1961,"YearRemodAdd":1961,"RoofStyle":"Gable","RoofMatl":"CompShg","Exterior1st":"VinylSd","Exterior2nd":"VinylSd","MasVnrType":"None","MasVnrArea":0.0,"ExterQual":"TA","ExterCond":"TA","Foundation":"CBlock","BsmtQual":"TA","BsmtCond":"TA","BsmtExposure":"No","BsmtFinType1":"Rec","BsmtFinSF1":468.0,"BsmtFinType2":"LwQ","BsmtFinSF2":144.0,"BsmtUnfSF":270.0,"TotalBsmtSF":882.0,"Heating":"GasA","HeatingQC":"TA","CentralAir":"Y","Electrical":"SBrkr","1stFlrSF":896,"2ndFlrSF":0,"LowQualFinSF":0,"GrLivArea":896,"BsmtFullBath":0.0,"BsmtHalfBath":0.0,"FullBath":1,"HalfBath":0,"BedroomAbvGr":2,"KitchenAbvGr":1,"KitchenQual":"TA","TotRmsAbvGrd":5,

## Test API With Request

In [36]:
# 1 - First run api 

In [40]:
base_url='http://127.0.0.1:5000/'

In [41]:
res=requests.get(base_url+"version")
res.json()

{'api_version': '0.2.0', 'model_version': '0.1'}

In [43]:
sample_data=pd.read_csv("test_data.csv")[:1]
sample_data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-121.49,38.58,52.0,569.0,405.0,509.0,367.0,0.9196,137500.0,INLAND


In [55]:
s=sample_data.to_json(orient="records")
s

'[{"longitude":-121.49,"latitude":38.58,"housing_median_age":52.0,"total_rooms":569.0,"total_bedrooms":405.0,"population":509.0,"households":367.0,"median_income":0.9196,"median_house_value":137500.0,"ocean_proximity":"INLAND"}]'

In [52]:
r=requests.post(base_url+'v1/predict',json=json.loads(s))


In [53]:
r.json()

{'return': [135112.023080916], 'state': 'ok'}

## Data Validation

In [378]:
sample_data=pd.read_csv("test_data.csv")[:12]
# drop target column
sample_data=sample_data.drop(columns="median_house_value")


In [379]:
sample_data

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
0,-121.49,38.58,52.0,569.0,405.0,509.0,367.0,0.9196,INLAND
1,-117.96,33.83,34.0,982.0,148.0,498.0,156.0,6.3214,<1H OCEAN
2,-118.28,33.68,8.0,2842.0,522.0,1624.0,510.0,3.7282,NEAR OCEAN
3,-121.94,37.56,15.0,5674.0,748.0,2412.0,714.0,8.3996,<1H OCEAN
4,-118.13,33.82,36.0,665.0,114.0,273.0,112.0,3.7321,<1H OCEAN
5,-117.25,33.93,8.0,10110.0,1761.0,5804.0,1703.0,4.2654,INLAND
6,-121.49,38.63,6.0,12197.0,2617.0,5634.0,2329.0,3.7449,INLAND
7,-118.52,34.19,37.0,1892.0,347.0,1039.0,343.0,4.8295,<1H OCEAN
8,-119.75,34.45,26.0,3578.0,677.0,1504.0,618.0,4.1375,NEAR OCEAN
9,-117.09,32.66,38.0,833.0,206.0,570.0,182.0,1.8333,NEAR OCEAN


In [380]:
sample_data.loc[4,"latitude"]="deneme"
sample_data.loc[0,"ocean_proximity"]=None

In [381]:
# convert json and python object
sample_json=json.loads(sample_data.to_json(orient="records"))

### pydantic

In [327]:
from pydantic import BaseModel,parse_obj_as,ValidationError

#### Examples

In [128]:
from pydantic import BaseModel
from pydantic import parse_obj_as



In [129]:
class User(BaseModel):
  name: str
  age: int

In [130]:
users = [
    {"name": "user1", "age": 15}, 
    {"name": "user2", "age": 28}
]


In [131]:
m = parse_obj_as(List[User], users)

In [132]:
m

[User(name='user1', age=15), User(name='user2', age=28)]

#### Try with real data 

**Input data validate**

In [382]:
class InputData(BaseModel):
 
    # A sample schema for input unit data
    
    longitude:float
    latitude:float
    housing_median_age:float
    total_rooms:float
    total_bedrooms:float
    population:float
    households:float
    median_income:float
    ocean_proximity:str

class InputList(BaseModel):
    
    # input data schema     
    data: List[InputData]

In [390]:
# check errors
errors=None
try:
    m=InputList(data=sample_json,many=True)
    
except ValidationError as e:
    errors=e.errors()

In [392]:
# delete error rows
if errors:
    # find errors index
    indexes=[i["loc"][1] for i in errors]
    #print(indexes)

    for index in sorted(indexes,reverse=True):
        del sample_json[index]

In [393]:
def validate_inputs(input_data):
    
    # Check input data and remove errors
    
    # check errors
    errors=None
    try:
        m=InputList(data=input_data,many=True)

    except ValidationError as e:
        errors=e.errors()
        
    # delete error rows
    if errors:
        # find errors index
        indexes=[i["loc"][1] for i in errors]
        #print(indexes)

        for index in sorted(indexes,reverse=True):
            del input_data[index]
            
    return input_data,errors

### Marshmallow 

In [338]:
class InputDataSchema(Schema):
    longitude:fields.Float()
    latitude:fields.Float()
    housing_median_age:fields.Float()
    total_rooms:fields.Float()
    total_bedrooms:fields.Float()
    population:fields.Float()
    households:fields.Float()
    median_income:fields.Float()
    ocean_proximity:fields.Str()
        
   

In [342]:
schema = InputDataSchema(strict=True, many=True)


In [341]:
schema.load(sample_json)

UnmarshalResult(data=[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}], errors={})

In [343]:
errors=None

try:
    schema.load(sample_json)
except ValidationError as exc:
    errors=exc.messages

In [77]:
sample_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           1 non-null      float64
 1   latitude            1 non-null      float64
 2   housing_median_age  1 non-null      float64
 3   total_rooms         1 non-null      float64
 4   total_bedrooms      1 non-null      float64
 5   population          1 non-null      float64
 6   households          1 non-null      float64
 7   median_income       1 non-null      float64
 8   ocean_proximity     1 non-null      object 
dtypes: float64(8), object(1)
memory usage: 200.0+ bytes


In [75]:
class InputData(BaseModel):
    lo

SyntaxError: invalid syntax (<ipython-input-75-4d67b6505afe>, line 1)

In [ ]:
class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: List[int] = []


external_data = {
    'id': '123',
    'signup_ts': '2019-06-01 12:22',
    'friends': [1, 2, '3'],
}
user = User(**external_data)
print(user.id)
#> 123
print(repr(user.signup_ts))
#> datetime.datetime(2019, 6, 1, 12, 22)
print(user.friends)
#> [1, 2, 3]
print(user.dict())
"""
{
    'id': 123,
    'signup_ts': datetime.datetime(2019, 6, 1, 12, 22),
    'friends': [1, 2, 3],
    'name': 'John Doe',
}
"""

In [71]:



class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: List[int] = []




In [90]:
Opti

typing.Optional

In [85]:
s={12,12}

In [81]:
s=(12,2)

In [87]:
type(s)

set

In [74]:
external_data = {
    'id': {1:2},
    'signup_ts': '2019-06-01 12:22',
    'friends': [1, 2, '3'],
}
user = User(**external_data)

ValidationError: 1 validation error for User
id
  value is not a valid integer (type=type_error.integer)

In [73]:
user

User(id=12, signup_ts=datetime.datetime(2019, 6, 1, 12, 22), friends=[1, 2, 3], name='John Doe')

In [64]:
from pydantic import ValidationError

try:
    User(signup_ts='broken', friends=[1, 2, 'not number'])
except ValidationError as e:
    print(e.json())

In [95]:
from pydantic import BaseModel


class User(BaseModel):
    name: str
    username: str
    password: int


user = User(name="Redowan Delowar", username="rednafi", password=[1,34])

print(user)

ValidationError: 1 validation error for User
password
  value is not a valid integer (type=type_error.integer)

###  API Request

In [9]:
res=requests.get("http://127.0.0.1:5000/version")

In [12]:
res.json()

{'api_version': '0.2.0', 'model_version': '0.1'}

In [13]:
post_json={"test1":1,"test2":2}


In [14]:
json.loads(json.dumps(post_json))

{'test1': 1, 'test2': 2}

In [15]:

res=requests.post("http://127.0.0.1:5000/test",json=json.loads(json.dumps(post_json)))

In [46]:
i=image.imread("test.jpg")

In [31]:
params = {'param0': 'param0', 'param1': 'param1'}
#arr = np.random.rand(10, 10)
#data = {'params': params, 'arr': arr.tolist()}
data={"params":params,"image":i.tolist()}


In [54]:
%%time
data = pickle.dumps(i)
res=requests.post("http://127.0.0.1:5000/test",data=data)

CPU times: user 5.72 ms, sys: 0 ns, total: 5.72 ms
Wall time: 6.12 ms


In [43]:
%%time
res=requests.post("http://127.0.0.1:5000/test",json=data)

CPU times: user 225 ms, sys: 11.4 ms, total: 236 ms
Wall time: 633 ms
